In [1]:
import numpy as np
import pandas as pd
# I get the data and sort it in MATLAB, and save the data as Excel file. Each file contains one-year data.

In [10]:
# The yearly data are from 2005 to 2015. There are 10 gourps(features in this year with return in next year).
# choose companies listed before 2005, and remove those company whose data is not complete. Then 1037 stocks are considered.
# I choose 13 factors(all fundmental factors) according to some analysis reports, including total market value, EPS, BPS
# 12-month swing, 100-week volitility, 3-year net income growth, 3-year revenue growth, 60-day price change, 
# 120-day price change, 240-day price change, operating income, total current assets, and account receivable.
data1=pd.read_excel(r'C:\Users\Fu\Documents\GitHub\Project\data1.xlsx')
data2=pd.read_excel(r'C:\Users\Fu\Documents\GitHub\Project\data2.xlsx')
data3=pd.read_excel(r'C:\Users\Fu\Documents\GitHub\Project\data3.xlsx')
data4=pd.read_excel(r'C:\Users\Fu\Documents\GitHub\Project\data4.xlsx')
data5=pd.read_excel(r'C:\Users\Fu\Documents\GitHub\Project\data5.xlsx')
data6=pd.read_excel(r'C:\Users\Fu\Documents\GitHub\Project\data6.xlsx')
data7=pd.read_excel(r'C:\Users\Fu\Documents\GitHub\Project\data7.xlsx')
data8=pd.read_excel(r'C:\Users\Fu\Documents\GitHub\Project\data8.xlsx')
data9=pd.read_excel(r'C:\Users\Fu\Documents\GitHub\Project\data9.xlsx')
data10=pd.read_excel(r'C:\Users\Fu\Documents\GitHub\Project\data10.xlsx')
index=pd.read_excel(r'C:\Users\Fu\Documents\GitHub\Project\index.xlsx')    # the market index(CSI 800 Index)
index

,CLOSE,PCT_CHG
0,898.7830,-8.650517
1,1948.9010,116.837768
2,5205.7370,167.111413
3,1824.4000,-64.954050
4,3715.3590,103.648268
5,3443.5760,-7.315121
6,2500.6520,-27.382117
7,2645.8580,5.806726
8,2589.3530,-2.135602
9,3839.3850,48.275843


In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

In [14]:
# in each year, use the current-year data as training data, and the next-year data as testing data.
# As we'll adjust the method after seeing the result, the last 2-year data is remianed for final prediction test.
accuracy=[]
accuracy_pca=[]
for i in range(1,8):     # choose 8 out of 10 years
    s_train="data"+str(i)
    data_train=eval(s_train)
    X_train,Y_train=data_train.iloc[:,1:14].values,data_train.iloc[:,-1].values
    s_test="data"+str(i+1)
    data_test=eval(s_test)
    X_test,Y_test=data_test.iloc[:,1:14].values,data_test.iloc[:,-1].values
    
    s=StandardScaler()
    X_train_std=s.fit_transform(X_train)
    X_test_std=s.fit_transform(X_test)
    
    Y_train=Y_train>index['PCT_CHG'][i]
    Y_test=Y_test>index['PCT_CHG'][i+1]
    
    svm_linear=SVC(kernel='linear',C=1.0,random_state=0)
    svm_linear.fit(X_train_std,Y_train)
    
    score1=svm_linear.score(X_test_std,Y_test)
    accuracy.append(score1)   # the accuracy result by SVM
    
    pipe_svm_pca = Pipeline([('scl', StandardScaler()),
                         ('pca', PCA(n_components=2)),
                    ('clf', SVC(kernel='linear', C=1.0, random_state=0))])
    pipe_svm_pca.fit(X_train_std,Y_train)
    score2=pipe_svm_pca.score(X_test_std,Y_test)
    accuracy_pca.append(score2)

[accuracy,accuracy_pca]

[[0.45033751205400191,
  0.61523625843780139,
  0.17550626808100289,
  0.38283510125361619,
  0.38572806171648988,
  0.94503375120540023,
  0.38090646094503378],
 [0.45130183220829317,
  0.61909353905496622,
  0.070395371263259399,
  0.38187078109932499,
  0.3741562198649952,
  0.95274831243973002,
  0.38090646094503378]]

In [ ]:
# However the answer is much different from MATLAB result.
#accuracy
#[0.446480231436837
#0.591128254580521
#0.661523625843780
#0.586306653809065
#0.469623915139826
#0.525554484088717
#0.641272902603664]   the mean accuracy is 0.5603, which is an acceptable pattern to set up a strategy.

In [15]:
print('SVM train accuracy : %.3f' %np.mean(accuracy))
print('SVM(with PCA) train accuracy : %.3f' %np.mean(accuracy_pca))

SVM train accuracy : 0.477
SVM(with PCA) train accuracy : 0.461


In [17]:
# To furthur test these factors. in each loop, eliminating one factor and calculate the test accuracy, if the accuracy 
# increases, the factor would be eliminated from the model.
accuracy_el=[]
for i in range(1,8):
    s_train="data"+str(i)
    data_train=eval(s_train)
    X_train,Y_train=data_train.iloc[:,1:14].values,data_train.iloc[:,-1].values
    s_test="data"+str(i+1)
    data_test=eval(s_test)
    X_test,Y_test=data_test.iloc[:,1:14].values,data_test.iloc[:,-1].values
    
    s=StandardScaler()
    X_train_std=s.fit_transform(X_train)
    X_test_std=s.fit_transform(X_test)
    
    Y_train=Y_train>index['PCT_CHG'][i]
    Y_test=Y_test>index['PCT_CHG'][i+1]
    accuracy_j=[]
    for j in range(0,13):
        X_train_std_el=np.array([[row[k] for k in range(0, 13) if k != j] for row in X_train_std])   # remove the jth factor's data
        X_test_std_el=np.array([[row[k] for k in range(0, 13) if k != j] for row in X_test_std]) 

        svm_linear=SVC(kernel='linear',C=1.0,random_state=0)
        svm_linear.fit(X_train_std_el,Y_train)
        score3=svm_linear.score(X_test_std_el,Y_test)
        accuracy_j.append(score3)
    accuracy_el.append(accuracy_j)

isredundant=np.mean(accuracy_el,axis=0)-np.mean(accuracy)>0
redundant=[]
for i in range(len(isredundant)):
    if isredundant[i]:
        redundant.append(i)
redundant

[2, 4, 5, 6, 7]

In [ ]:
# After test, only 8 faactors left,total market value, BPS, 12-month swing, 100-week volitility,
# 60-day price change, 120-day price change, 240-day price change,  and account receivable.

In [18]:
# final (prediction) test
accuracy_rest=[]
for i in range(1,10):
    s_train="data"+str(i)
    data_train=eval(s_train)
    X_train,Y_train=data_train.iloc[:,1:14].values,data_train.iloc[:,-1].values
    s_test="data"+str(i+1)
    data_test=eval(s_test)
    X_test,Y_test=data_test.iloc[:,1:14].values,data_test.iloc[:,-1].values
    
    s=StandardScaler()
    X_train_std=s.fit_transform(X_train)
    X_test_std=s.fit_transform(X_test)
    
    Y_train=Y_train>index['PCT_CHG'][i]
    Y_test=Y_test>index['PCT_CHG'][i+1]
    X_train_std_rest=np.array([[row[k] for k in range(0, 13) if (not k in redundant)] for row in X_train_std]) 
    X_test_std_rest=np.array([[row[k] for k in range(0, 13) if (not k in redundant)] for row in X_test_std]) 

    svm_linear=SVC(kernel='linear',C=1.0,random_state=0)
    svm_linear.fit(X_train_std_rest,Y_train)
    score4=svm_linear.score(X_test_std_rest,Y_test)
    accuracy_rest.append(score4)
accuracy_rest

[0.45033751205400191,
 0.62680810028929601,
 0.17839922854387658,
 0.38187078109932499,
 0.37608486017357762,
 0.95178399228543875,
 0.38090646094503378,
 0.47637415621986501,
 0.38283510125361619]

In [19]:
print('SVM(with remained factors) prediction accuracy : %.3f' %np.mean(accuracy_rest))
# The result is much different from MATLAB(0.567), too.

SVM(with remained factors) prediction accuracy : 0.467


In [ ]:
# From the result, the predition accuracy is stably higher than 50%(in MATLAB). Thus we can construct a investing strategy
# Just test a direct and easy one. Buy all the stocks that is predicted to be better than the index in euqal weight.
# the calculation and result are in MATLAB file.